In [1]:
#!/usr/bin/env python
from __future__ import print_function

from datetime import datetime
from collections import OrderedDict
import random
import os
import pandas as pd

import axioma
from axioma.assetset import AssetSet, ActionEntry
from axioma.account import Account
from axioma.accountgroup import AccountGroup
from axioma.workspace import DerbyProvider, Workspace
from axioma.workspace_element import ElementType
from axioma.costmodel import CostModel, CostStructure
from axioma.group import Group, Benchmark, Unit
from axioma.contentbuilder_group import ContentBuilderBenchmark, ContentBuilderGroup
from axioma.riskmodel import RiskModel
from axioma.strategy import Strategy, Objective, Target, Scope, MarketImpactType, PenaltyType
from axioma.rebalancing import Rebalancing, RebalancingStatus
from axioma.multiportfolio_rebalancing import MultiPortfolioRebalancing
from axioma.workspace_element import ElementType
from axioma.metagroup import Metagroup, DynamicMetagroup
from axioma.analytics import Analytics
import axioma.workspace_io as handler




In [2]:
axioma.ENDPOINT="http://localhost:8085/axioma-websrv"

In [18]:
from re import A, S
from numpy import maximum

if 'ws' in locals():
    ws.destroy()
else:
    print("The variable 'ws' does not exist.")

dates = ["2024-10-01","2024-10-02","2024-10-03"]

model = "WW51AxiomaSH"

# iterate through dates in dates list

############################## CREATE WORKSPACE
i=1
date = datetime.strptime(dates[i], '%Y-%m-%d').date()
next_period_date = datetime.strptime(dates[i+1], '%Y-%m-%d').date()
axioma_data_dir = r'C:/Users/jwbpa/Box/Altisma_Data/AxiomaDownloader-3.3.0-with-jre64/output/extractedFiles/database/${yyyy}/${mm}/'
db_provider = DerbyProvider(axioma_data_dir,
                            risk_models=model,
                            include_composites=True,
                            next_period_date=next_period_date,
                            returns_type="Gross Return")

ws = Workspace(f"AltismaWorkspace_{dates[i]}", date, data_provider=db_provider)

############################## LOAD DATA

ac_df = pd.read_csv(f"C:/Users/jwbpa/Box/Altisma_Data/Axioma/Example_Optimization/AlphaCapture_Optimized_{dates[i]}.csv", index_col="Name")
ac_account = Account(workspace = ws, 
        identity = "AC_Input", 
        holdings=ac_df.to_dict()["Shares"], 
        asset_map="Ticker Map")

ae_df = pd.read_csv(f"C:/Users/jwbpa/Box/Altisma_Data/Axioma/Example_Optimization/big_portfolio_{dates[i-1]}.csv", index_col="Name")
ae_account = Account(workspace = ws, 
        identity = "AE_Input", 
        holdings=ae_df.to_dict()["Shares"], 
        asset_map="Ticker Map")



net_account=AccountGroup(workspace=ws, identity = "AE_AC_AGGREGATE")
net_account.add_account(ac_account)
net_account.add_account(ae_account)


net_long_value = net_account.get_long_value(price_group="Price", exclude_futures=False)
net_short_value = net_account.get_short_value(price_group="Price", exclude_futures=False)
net_cash_value = net_account.get_total_cash_value(price_group="Price")
net_gross_value=net_long_value+net_short_value+net_cash_value

ac_long_value = ac_account.get_long_value(price_group="Price", exclude_futures=False)
ac_short_value = ac_account.get_short_value(price_group="Price", exclude_futures=False)
ac_cash_value = ac_account.get_total_cash_value(price_group="Price")
ac_gross_value=ac_long_value+ac_short_value+ac_cash_value

ae_long_value = ae_account.get_long_value(price_group="Price", exclude_futures=False)
ae_short_value = ae_account.get_short_value(price_group="Price", exclude_futures=False)
ae_cash_value = ae_account.get_total_cash_value(price_group="Price")
ae_gross_value=ae_long_value+ae_short_value+ae_cash_value


# Benchmark
handler.load_assets_from_data_provider(workspace=ws, asset_names=["SPY"], asset_map="Ticker Map")
# SPY benchmark is referenced via its composition; explicit asset load is unnecessary
spy_benchmark = ContentBuilderBenchmark(workspace=ws, identity = "SPY_Benchmark", expression = "'Composition of 37P4NKR33'*1")


## Content Builder Attributes
# Content Builder for 60-Day MDV
Inv_60_Day_MDV = ContentBuilderGroup(workspace=ws, identity = "Inv_60_Day_MDV", expression = "(1/'60-Day MDV')")

# Content Builder for Account Currency
Account_Currency = ContentBuilderGroup(workspace=ws, identity = "Account_Currency", expression = "portfolioAsCurrency('AC_Input')*1")

In [25]:
try:
    if 'strategy' in locals() and strategy is not None:
        try:
            strategy.destroy()
        except Exception as e:
            # AxiomaError when server object is already gone; ignore that case
            if "ResourceNotFoundException" not in str(e):
                raise
finally:
    if 'strategy' in locals():
        del strategy


############################## DEFINE STRATEGY
strategy = Strategy(workspace = ws, 
                    identity = "Strategy", 
                    allow_shorting=True, 
                    allow_crossover=False, 
                    allow_grandfathering=True,
                    enable_constraint_hierarchy=True)
strategy.set_local_universe(local_universe=list(ac_account.get_holdings().keys()) + ["CSH_USD__"])

############################## DEFINE OBJECTIVE TERMS
ar_term = axioma.strategy.create_risk_term(strategy = strategy, 
                                            identity = "activeRisk", 
                                            benchmark=Account_Currency,
                                            factor_weight=0, 
                                            risk_model=model,
                                            specific_weight=1.0,
                                            asset_set="MASTER",
                                            qualification=ac_account
                                            )
tc_term = axioma.strategy.create_market_impact_term(strategy = strategy, 
                                                    identity = "marketImpact",
                                                    buy_impact_group=Inv_60_Day_MDV,
                                                    sell_impact_group=Inv_60_Day_MDV,
                                                    market_impact_type=MarketImpactType.Quadratic
                                                    )
############################## DEFINE OBJECTIVE FUNCTION
terms = OrderedDict()
terms[ar_term] = 1.0
terms[tc_term] = 1.0
obj_fx = Objective(strategy = strategy, 
                    identity = "Objective", 
                    terms=terms, 
                    target=Target.Minimize, 
                    active=True)
############################## DEFINE CONSTRAINTS

# constraint 0
do_not_trade_ae=axioma.strategy.create_limit_trade_constraint(strategy = strategy,
                                                              identity = "doNotTradeAE",
                                                              minimum=0,
                                                              maximum=0,
                                                              scope=Scope.Asset,
                                                              unit=Unit.Percent
)
# add multiselection qualification based on ae_df
do_not_trade_ae.add_selection(element_type=ElementType.AssetSet, element="MASTER", qualification=ae_account)


# constraint 1
match_benchmark_market_exposure=axioma.strategy.create_limit_holding_constraint(
strategy = strategy,
identity = "matchBenchmarkMarketExposure",
minimum=0,
maximum=0,
benchmark=spy_benchmark,
scope=Scope.Selection,
unit=Unit.Percent
)

# add Market Intercept factor 
#match_benchmark_market_exposure.add_selection(element_type=ElementType.Group, element=f"{model}.Market Intercept")
market_mgp = ws.get_metagroup(f"{model}.Market")
for group in market_mgp.get_groups():
    match_benchmark_market_exposure.add_selection(element_type=ElementType.Group, element=group)





# constraint 2
match_benchmark_style_exposure=axioma.strategy.create_limit_holding_constraint(strategy = strategy,
                                                                                 identity = "matchBenchmarkStyleExposure",
                                                                                 minimum=-0.1,
                                                                                 maximum=0.1,
                                                                                 benchmark=spy_benchmark,
                                                                                 scope=Scope.Member,
                                                                                 unit=Unit.Percent
                                                                                 )

# add style factors to multiselection
#match_benchmark_style_exposure.add_selection(element_type=ElementType.Metagroup, element=f"{model}.Style")
style_mgp = ws.get_metagroup(f"{model}.Style")
for group in style_mgp.get_groups():
    match_benchmark_style_exposure.add_selection(element_type=ElementType.Group, element=group)

In [ ]:




# constraint 3
match_benchmark_sector_exposure=axioma.strategy.create_limit_holding_constraint(strategy = strategy,
                                                                                 identity = "matchBenchmarkSectorExposure",
                                                                                 minimum=-0.1,
                                                                                 maximum=0.1,
                                                                                 benchmark=spy_benchmark,
                                                                                 scope=Scope.Selection,
                                                                                 unit=Unit.Percent
                                                                                 )

# add sector factors to multiselection except real estate and consumer discretionary
match_benchmark_sector_exposure.add_selection(element_type=ElementType.Metagroup, element=f"{model}.Communication Services")
match_benchmark_sector_exposure.add_selection(element_type=ElementType.Group, element=f"{model}.Medium-Term Momentum")
match_benchmark_sector_exposure.add_selection(element_type=ElementType.Group, element=f"{model}.Residual Volatility")





# constraint 4
match_benchmark_realestate_sector_exposure=axioma.strategy.create_limit_holding_constraint(strategy = strategy,
                                                                                 identity = "matchBenchmarkRealestateSectorExposure",
                                                                                 minimum=-0.1,
                                                                                 maximum=0.1,
                                                                                 benchmark=spy_benchmark,
                                                                                 scope=Scope.Selection,
                                                                                 unit=Unit.Percent
                                                                                 )

# add real estate  to multiselection
match_benchmark_realestate_sector_exposure.add_selection(element_type=ElementType.Asset, element=spy_benchmark.index)



# constraint 5
match_benchmark_consumerdiscretionary_sector_exposure=axioma.strategy.create_limit_holding_constraint(strategy = strategy,
                                                                                 identity = "matchBenchmarkConsumerDiscretionarySectorExposure",
                                                                                 minimum=-0.1,
                                                                                 maximum=0.1,
                                                                                 benchmark=spy_benchmark,
                                                                                 scope=Scope.Selection,
                                                                                 unit=Unit.Percent
                                                                                 )

# add consumer discretionary to multiselection
match_benchmark_consumerdiscretionary_sector_exposure.add_selection(element_type=ElementType.Asset, element=spy_benchmark.index)




# constraint 6
match_benchmark_aggregate_exposure=axioma.strategy.create_limit_holding_constraint(strategy = strategy,
                                                                                 identity = "matchBenchmarkAggregateExposure",
                                                                                 minimum=-0.5,
                                                                                 maximum=0.5,
                                                                                 benchmark=spy_benchmark,
                                                                                 scope=Scope.Aggregate,
                                                                                 unit=Unit.Percent
                                                                                 )

# add all factors to multiselection
rm = ws.get_risk_model(model)
for factor in rm.get_factor_names():
    match_benchmark_aggregate_exposure.add_selection(element_type=ElementType.Group, element=f"{model}.{factor}")




# constraint 7
budget=axioma.strategy.create_budget_constraint(strategy = strategy,
                                                identity = "budget",
                                                qualification=ac_account,
                                                use_budget_value=True
                                                )




# constraint 8
sum_squared_weights=axioma.strategy.create_limit_model_deviation_constraint(strategy = strategy,
                                                                            identity = "sumSquaredWeights",
                                                                            maximum=25,
                                                                            penalty=4000,
                                                                            penalty_type=PenaltyType.ScaledQuadratic,
                                                                            scope=Scope.Selection,
                                                                            unit=Unit.Percent
                                                                            )

# add NON-CASH ASSETS to selection
sum_squared_weights.add_selection(element_type=ElementType.AssetSet, element="NON-CASH ASSETS")


############################## SET CONSTRAINT HIERARCHY CONSTRAINTS

strategy.set_constraint_hierarchy({"matchBenchmarkRealestateSectorExposure" : 1})



AttributeError: 'function' object has no attribute 'keys'

In [ ]:

############################## CREATE REBALANCING
rebal = MultiPortfolioRebalancing(workspace = ws, 
                    identity="BetaCompletion_Rebalancing", 
                    strategy=strategy
                    )

rebal.add_account(ac_account)
rebal.add_account(ae_account)

rebal.set_reference_size(net_gross_value)
#rebal.set_budget_size(1000000)
#rebal.set_benchmark_size(1000000)


rebal.set_rebalancing_defaults(round_lot_size=1,
                               use_cash_for_roundlot=True,
                               risk_model=model)

############################## SOLVE REBALANCING
sol = rebal.solve()
print(sol.get_status())
if sol.get_status()==RebalancingStatus.SolutionFound or sol.get_status()==RebalancingStatus.RelaxedSolutionFound:
    fh = sol.get_final_holdings(asset_map = "Ticker Map")
#sol.get_final_holdings(asset_map=None)
ae_fh=sol.get_final_account_holdings_for_account(ae_account, asset_map="Ticker Map")
ac_fh=sol.get_final_account_holdings_for_account(ac_account, asset_map="Ticker Map")

In [ ]:


############################## CALCULATE ANALYTICS - THESE ARE EXAMPLES OF SOME AVAILABLE ANALYTICS
# create Analytics object. This will be used to calculate analytics
analyzer = Analytics(workspace = ws, price_group="Price", asset_map="Ticker Map")

# compute active holdings of final holdings
ah = analyzer.compute_active_holdings(holdings=fh, benchmark=spy_benchmark, reference_value=rebal.get_reference_size())

# calculate active exposures -> you divide df by reference size to convert from currency values to decimal
ah_exposures = pd.DataFrame.from_dict({"Exposure":analyzer.compute_factor_exposures(risk_model=model,holdings=ah)})/rebal.get_reference_size()

# compute tracking error.
tracking_error = analyzer.compute_factors_risk(risk_model=model,holdings=ah)/rebal.get_reference_size()*100


########################## Find Idio Return and Risk of final solution next day.
# Factor return
factor_return_dollar = analyzer.compute_factor_return_contributions(risk_model=model, 
                                                          metagroup = f"{model}.Period Returns",
                                                          holdings=ah)

# Total return
total_return_dollar =analyzer.compute_expected_return(alphas="Period Return", holdings=ah)

# Idio return
idio_return_dollar = total_return_dollar - factor_return_dollar

# Convert dollar to %
idio_return_percent = idio_return_dollar / rebal.get_reference_size()*100

# Factor risk
factor_risk_dollar = analyzer.compute_factors_risk(risk_model=model, holdings=ah)

# Total risk
total_risk_dollar = analyzer.compute_total_risk(risk_model=model, holdings=ah)

# Idio risk
idio_risk_dollar = analyzer.compute_specific_risk(risk_model=model, holdings=ah)

# Convert dollar to %
idio_risk_percent = idio_risk_dollar / rebal.get_reference_size()*100




# write workspace to .wsp file and release license token
ws.write(os.getcwd(),file_name=f"workspace_on_{dates[i]}.wsp",save_reference=False)
ws.destroy()
